# Day 4 Scratchcards

The gondola takes you up. Strangely, though, the ground doesn't seem to be coming with you; you're not climbing a mountain. As the circle of Snow Island recedes below you, an entire new landmass suddenly appears above you! The gondola carries you to the surface of the new island and lurches into the station.

As you exit the gondola, the first thing you notice is that the air here is much warmer than it was on Snow Island. It's also quite humid. Is this where the water source is?

The next thing you notice is an Elf sitting on the floor across the station in what seems to be a pile of colorful square cards.

"Oh! Hello!" The Elf excitedly runs over to you. "How may I be of service?" You ask about water sources.

"I'm not sure; I just operate the gondola lift. That does sound like something we'd have, though - this is Island Island, after all! I bet the gardener would know. He's on a different island, though - er, the small kind surrounded by water, not the floating kind. We really need to come up with a better naming scheme. Tell you what: if you can help me with something quick, I'll let you borrow my boat and you can go visit the gardener. I got all these scratchcards as a gift, but I can't figure out what I've won."

The Elf leads you over to the pile of colorful cards. There, you discover dozens of scratchcards, all with their opaque covering already scratched off. Picking one up, it looks like each card has two lists of numbers separated by a vertical bar (|): a list of winning numbers and then a list of numbers you have. You organize the information into a table (your puzzle input).

As far as the Elf has been able to figure out, you have to figure out which of the numbers you have appear in the list of winning numbers. The first match makes the card worth one point and each match after the first doubles the point value of that card.

For example:

Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53\
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19\
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1\
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83\
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36\
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11

In the above example, card 1 has five winning numbers (41, 48, 83, 86, and 17) and eight numbers you have (83, 86, 6, 31, 17, 9, 48, and 53). Of the numbers you have, four of them (48, 83, 17, and 86) are winning numbers! That means card 1 is worth 8 points (1 for the first match, then doubled three times for each of the three matches after the first).

Card 2 has two winning numbers (32 and 61), so it is worth 2 points.
Card 3 has two winning numbers (1 and 21), so it is worth 2 points.
Card 4 has one winning number (84), so it is worth 1 point.
Card 5 has no winning numbers, so it is worth no points.
Card 6 has no winning numbers, so it is worth no points.
So, in this example, the Elf's pile of scratchcards is worth 13 points.

Take a seat in the large pile of colorful cards. How many points are they worth in total?

In [527]:
import pandas as pd
import numpy as np
import os


with open('input.txt', 'r') as file:
    input = file.read()

# input = '''Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
# Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
# Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
# Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
# Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
# Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
# '''

In [528]:
df = pd.DataFrame({'Data': input.split('\n')})
df[['Card', 'Tickets']] = df['Data'].str.split(':', expand=True)
df = df.drop(columns='Data')

df[['WinningNumbers', 'MyNumbers']] = df['Tickets'].str.split('|', expand=True)
df = df.drop(columns='Tickets').dropna()

df['Card'] = df.index + 1

cards = df[['Card']].drop_duplicates()

winning_numbers = df['WinningNumbers'].str.split(' ', expand=True)
my_numbers = df['MyNumbers'].str.split(' ', expand=True)

def clean_number_tickets(df):
    df['Card'] = df.index + 1
    df = df.melt(id_vars='Card', value_vars=df.columns[:-1]).sort_values(['Card', 'variable']).fillna('')
    df = df.loc[df['value'].str.isdigit()].reset_index(drop=True)
    df['variable'] = df.groupby(['Card']).cumcount()
    return df

my_numbers = clean_number_tickets(my_numbers)
winning_numbers = clean_number_tickets(winning_numbers)

my_numbers.columns = ['Card', 'm_Order', 'm_Number']
winning_numbers.columns = ['Card', 'w_Order', 'w_Number']

df = winning_numbers.merge(my_numbers).astype(int)
df = df.query("w_Number == m_Number").sort_values(['Card', 'w_Order', 'm_Order'], ascending=True)
df['MatchCounter'] = df.groupby(['Card']).cumcount() + 1
# df = df.groupby('Card')['MatchCounter'].max().reset_index()

#df['Value'] = 2**(df['MatchCounter']-1)

df = df.merge(cards, how='outer').fillna(0)

# df['Value'].sum()

Answer - 20407

## Part 2
Just as you're about to report your findings to the Elf, one of you realizes that the rules have actually been printed on the back of every card this whole time.

There's no such thing as "points". Instead, scratchcards only cause you to win more scratchcards equal to the number of winning numbers you have.

Specifically, you win copies of the scratchcards below the winning card equal to the number of matches. So, if card 10 were to have 5 matching numbers, you would win one copy each of cards 11, 12, 13, 14, and 15.

Copies of scratchcards are scored like normal scratchcards and have the same card number as the card they copied. So, if you win a copy of card 10 and it has 5 matching numbers, it would then win a copy of the same cards that the original card 10 won: cards 11, 12, 13, 14, and 15. This process repeats until none of the copies cause you to win any more cards. (Cards will never make you copy a card past the end of the table.)

This time, the above example goes differently:

Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53\
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19\
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1\
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83\
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36\
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11

Card 1 has four matching numbers, so you win one copy each of the next four cards: cards 2, 3, 4, and 5.

Your original card 2 has two matching numbers, so you win one copy each of cards 3 and 4.

Your copy of card 2 also wins one copy each of cards 3 and 4.

Your four instances of card 3 (one original and three copies) have two matching numbers, so you win four copies each of cards 4 and 5.

Your eight instances of card 4 (one original and seven copies) have one matching number, so you win eight copies of card 5.

Your fourteen instances of card 5 (one original and thirteen copies) have no matching numbers and win no more cards.

Your one instance of card 6 (one original) has no matching numbers and wins no more cards.

Once all of the originals and copies have been processed, you end up with 1 instance of card 1, 2 instances of card 2, 4 instances of card 3, 8 instances of card 4, 14 instances of card 5, and 1 instance of card 6. In total, this example pile of scratchcards causes you to ultimately have 30 scratchcards!

Process all of the original and copied scratchcards until no more scratchcards are won. Including the original set of scratchcards, how many total scratchcards do you end up with?

In [529]:
df['MatchingCards'] = np.where(df['MatchCounter'] != 0, df['MatchCounter'] + df['Card'], 0)
df['CardList'] = df['Card'].astype(str)

df

,Card,w_Order,w_Number,m_Order,m_Number,MatchCounter,MatchingCards,CardList
0,1,0.0,77.0,19.0,77.0,1.0,2.0,1
1,1,1.0,45.0,12.0,45.0,2.0,3.0,1
2,1,2.0,9.0,14.0,9.0,3.0,4.0,1
3,1,3.0,81.0,16.0,81.0,4.0,5.0,1
4,1,4.0,96.0,4.0,96.0,5.0,6.0,1
...,...,...,...,...,...,...,...,...
823,154,0.0,0.0,0.0,0.0,0.0,0.0,154
824,164,0.0,0.0,0.0,0.0,0.0,0.0,164
825,167,0.0,0.0,0.0,0.0,0.0,0.0,167
826,174,0.0,0.0,0.0,0.0,0.0,0.0,174


In [530]:
# for card in cards['Card']:
#     temp = df.query("Card == @card")
#     for matching_card in temp['MatchingCards']:
#         temp2 = df.query("Card == @matching_card").copy()
#         temp2['CardList'] = temp2['CardList'] + '_' + temp2['MatchingCards'].astype(str)
#         df = pd.concat([df, temp2])

In [531]:
part2_df = df[['Card', 'MatchingCards', 'CardList']].copy()
part2_df['MatchingCards'] = part2_df['MatchingCards'].astype(int)
part2_df = part2_df.astype(str)
part2_df

,Card,MatchingCards,CardList
0,1,2,1
1,1,3,1
2,1,4,1
3,1,5,1
4,1,6,1
...,...,...,...
823,154,0,154
824,164,0,164
825,167,0,167
826,174,0,174


In [532]:
class Card:
    def __init__(self, id, orig, matching_cards):
        self.id = id
        self.orig = orig
        self.matching_cards = matching_cards

    def __str__(self):
        return f'({self.id}, {self.orig}): {self.matching_cards}'
    
    def __repr__(self):
        return f'({self.id}, {self.orig}): {self.matching_cards}'

card_list = []

for card in part2_df['Card'].unique():
    card_list.append(
        Card(
            card,
            card,
            matching_cards=part2_df.query('Card==@card')['MatchingCards'].to_list()))

print(card_list)

[(1, 1): ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11'], (2, 2): ['3', '4', '5'], (3, 3): ['4'], (4, 4): ['5'], (5, 5): ['6', '7', '8', '9', '10', '11', '12', '13', '14', '15'], (7, 7): ['8'], (10, 10): ['11', '12', '13', '14', '15', '16', '17'], (11, 11): ['12', '13', '14', '15', '16', '17'], (12, 12): ['13', '14'], (13, 13): ['14', '15', '16', '17', '18'], (14, 14): ['15', '16', '17', '18', '19'], (15, 15): ['16'], (16, 16): ['17', '18', '19'], (17, 17): ['18', '19', '20'], (18, 18): ['19', '20'], (21, 21): ['22', '23', '24', '25', '26', '27', '28', '29', '30', '31'], (22, 22): ['23', '24', '25', '26', '27', '28', '29', '30', '31', '32'], (23, 23): ['24', '25', '26', '27', '28', '29', '30', '31', '32', '33'], (24, 24): ['25', '26', '27', '28', '29', '30', '31', '32', '33', '34'], (25, 25): ['26', '27', '28', '29', '30', '31', '32', '33', '34', '35'], (26, 26): ['27', '28', '29', '30', '31', '32', '33', '34', '35', '36'], (27, 27): ['28', '29'], (28, 28): ['29', '30', '31', '32'

In [533]:
counter = 0

while counter < len(card_list):
    current_card = card_list[counter]
    orig_card = part2_df.query('Card == @current_card.orig')
    for card in orig_card['MatchingCards']:
        new_card = Card(
            id=f'{current_card.id}_{card}',
            orig=card,
            matching_cards=part2_df.query('Card == @card')['MatchingCards'].to_list()
        )
        if len(new_card.matching_cards) > 0:
            card_list.append(new_card)

    counter+=1

KeyboardInterrupt: 